In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [19]:
#!pip install pgmpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.2 MB/s eta 0:00:0000:0100:01


In [40]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
import numpy as np
from pgmpy.models import BayesianNetwork

In [125]:
NCmodel = BayesianModel()
NCmodel.add_node("ZipCode")
NCmodel.add_node("HaveCar")
NCmodel.add_node("PerCapitaIncome")
NCmodel.add_node("Havejob")
NCmodel.add_node("CommuteTime")
NCmodel.add_node("BusRidership")
NCmodel.add_node("Race")
NCmodel.add_node("UnemploymentRate")
NCmodel.add_node("Weekday")



NCmodel.add_edge("ZipCode", "HaveCar")
NCmodel.add_edge("ZipCode", "BusRidership")
NCmodel.add_edge("ZipCode", "UnemploymentRate")
NCmodel.add_edge("HaveCar", "PerCapitaIncome")
NCmodel.add_edge("HaveCar", "Havejob")
NCmodel.add_edge("PerCapitaIncome", "Havejob")

NCmodel.add_edge("Havejob", "BusRidership")
NCmodel.add_edge("UnemploymentRate","Havejob")
NCmodel.add_edge("Weekday","Havejob")

NCmodel.add_edge("CommuteTime", "BusRidership")
NCmodel.add_edge("Race", "BusRidership")
NCmodel.add_edge("Race", "ZipCode")
NCmodel.add_edge("UnemploymentRate", "Havejob")
NCmodel.add_edge("UnemploymentRate", "BusRidership")
NCmodel.add_edge("Weekday", "Havejob")



/Users/aaronl/opt/anaconda3/lib/python3.9/site-packages/pgmpy/models/BayesianModel.py:8: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  warnings.warn(


In [130]:
cpd_Race = TabularCPD('Race', variable_card=3,values=[[.8294],[.1327],[.0379]])
cpd_ZipCode = TabularCPD ('ZipCode', variable_card=3, values=[[.1775], [.2871], [.5354]],evidence=['Race'], evidence_card=[1])
cpd_CommuteTime = TabularCPD('CommuteTime', variable_card=4, values=[[.042], [.0661],[.3854], [.5065]])
cpd_Havejob = TabularCPD(variable='Havejob', variable_card=2, values=[[.29], [.71]], evidence=["PerCapitaIncome","HaveCar","Weekday","UnemploymentRate"], evidence_card=[1,1,1,1])
cpd_Weekday = TabularCPD('Weekday', variable_card=7, values=[[.1429], [.1429],[.1429], [.1429], [.1429], [.1429], [.1429]], evidence=['Havejob'] , evidence_card=[1])
cpd_HaveCar = TabularCPD('HaveCar', variable_card=2, values=[[.605], [.395]],evidence=['ZipCode'], evidence_card=[1])
cpd_PerCapitaIncome = TabularCPD ('PerCapitaIncome', variable_card=2, values=[[.7237], [.2763]],evidence=['HaveCar'],evidence_card=[1])
cpd_UnemploymentRate = TabularCPD ('UnemploymentRate', variable_card=2, values=[[.8881], [.1119]],evidence=["Havejob"], evidence_card=[1])

cpd_BusRidership = TabularCPD(variable='BusRidership', variable_card=2, values=[[.0], [.100]], evidence=["CommuteTime","HaveJob","ZipCode","Race"],evidence_card=[1,1,1,1])




NCmodel.add_cpds(cpd_CommuteTime, cpd_Weekday, cpd_HaveCar, cpd_PerCapitaIncome, cpd_UnemploymentRate, cpd_ZipCode, cpd_BusRidership, cpd_Havejob)


TypeError: object of type 'NoneType' has no len()

In [129]:
NCmodel.check_model()


ValueError: CPD associated with BusRidership doesn't have proper parents associated with it.